In [ ]:
import os 
import librosa
import numpy as np
import string as strp

In [ ]:
data_dir = '/Users/felipefelix/USP/tcc/dataset/exp/'

def generate_local_features(file_dir, feat_name, feat_func, y, **kwargs):
    print(kwargs)

    output_file = file_dir + '.' + feat_name + '.txt'

    if not os.path.isfile(output_file):
        print('generating {} for {}...'.format(feat_name, file_dir))
        feature = feat_func(y=y, **kwargs)
        np.savetxt(output_file, feature)
    else:
        print('{} already exists.'.format(output_file))
            

for subdir, dirs, files in os.walk(data_dir):
    for file in files:
        if file[0] != '.' and file.split('.')[-1] == 'mp3':
#            print('{} {} {}'.format(file, dirs, subdir))
            file_dir = subdir + '/' + file
            print('Loading {}...'.format(file_dir))
        
            y, sr = librosa.load(file_dir)
            
            generate_local_features(file_dir, 'rmse', librosa.feature.rmse, y)
            
            generate_local_features(file_dir, 'stft', librosa.core.stft, y)
            
            generate_local_features(file_dir, 'mfcc', librosa.feature.mfcc, y)
            
#            feature = librosa.feature.rmse(y = y)
#            np.savetxt(subdir + '/' + file + '.' + 'rmse', feature)
            

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm